This code is used to generate a set of races using the a race scheduling algorithm patterned after the Stearns (i.e., __Chaotic__) race scheduling method. It only works for a 2 lane raingutter regatta or pinewood derby race. The criteria are simple: make sure that each boy races the same number of times, make sure that boys never race in 2 consecutive races, and make sure that no 2 boys races each other more than once, and make sure that boys race the same number of times in each of the 2 lanes (all while being as random as possible).

In [61]:
from collections import Counter
import random

def hasConsecutiveRepeats(boys):
    last = ""
    for b in boys:
        if b == last:
            return True
        last = b
    return False

def sortedTuple(b1, b2):
    if b1 < b2:
        return (b1,b2)
    return (b2,b1)

# Depth-first search for a lane0 configuration that "works"
def fillInLane1(lane0, c, pairs, i, lane1):
    cands = [k for k,count in c.items() if count > 0]
    random.shuffle(cands)
    llane1 = lane1
    for candBoy in cands:
        p = sortedTuple(candBoy, lane0[i])
        if candBoy == lane0[i] or ((i > 0 and candBoy == lane0[i-1])
                                    or (i>0 and candBoy == llane1[-1])
                                    or (i< (len(lane0)-1) and candBoy == lane0[i+1])
                                    or p in pairs):
            pass
        else:
            # Build new frontier state to push onto the stack
            llane1.append(candBoy)
            
            if len(llane1) != i+1:
                print("i = {}, but len(lane1) = {}".format(i, len(lane1)))
            # If we have added the last boy and fulfilled our destiny, return lane1
            c[candBoy] -= 1
            
            pairs.add(p)
            if(len(lane0) == len(llane1)):
                return llane1
            e = []
            e.extend(llane1)
            l1 = fillInLane1(lane0, c, pairs, i+1, e)
            if l1:
                return l1

            llane1 = llane1[:-1]
            c[candBoy] += 1
            pairs.remove(p)
    return []

def fillInLane2(lane0, lane1, boys, numRacesEach, lane2):
    cands = boys * numRacesEach
    random.shuffle(cands)
    
    isGood = False
    while not isGood:
        isGood = True
        for i in range(0, len(cands)):
            candBoy = cands[i]
            # Check each item for a bad state, if there is a problem,
            # randomly swap until there aren't any more problems
            if (candBoy == lane0[i] or candBoy == lane1[i]
                                    or (i > 0 and (candBoy == lane0[i-1] or candBoy == lane1[i-1] or candBoy == cands[i-1]))
                                    or (i< (len(lane0)-1) and (candBoy == lane0[i+1] or candBoy == lane1[i+1]))):
                # Swap the "bad" candidate with a random location
                r = random.randint(0, len(cands)-1) 
                t = cands[i]
                cands[i] = cands[r]
                cands[r] = t
                isGood = False
    return cands
    
def buildRaces(boys, numRacesEach=3):
    """numRacesEach is the number of times the boys will race in
    each lane."""
    pairs = set()
    lane0 = boys * numRacesEach
    random.shuffle(lane0)
    lane1 = []
    # Get a shuffled list of boys for the lane 0 contestants
    while hasConsecutiveRepeats(lane0):
        random.shuffle(lane0)
    c = Counter(lane0)
        
    lane1 = fillInLane1(lane0, c, pairs, 0, lane1)
    
    lane2 = []
    lane2 = fillInLane2(lane0, lane1, boys, numRacesEach, lane2)

    print(Counter(lane0))
    print(Counter(lane1))
    print(Counter(lane2))

    return(list(zip(lane0, lane1, lane2)))

Type the names of the boys inside the quotation marks. Use commas to separate the names, don't include spaces unless the space is part of the boy's name.

In [69]:
boys = "Danny,Ian,David,Stirling,Braxton,Edgar,Ryan,Zach,Noah,Michael,Torrin".split(",")

The output of the following command will be the list of races. The races are listed as pairs of boys names "(A,B)" where A is the name of the boy that should use Lane 0 and B is the name of the boy who should use Lane 1. You can change the second parameter of buildRaces to a higher number if you would like, though there is a theoretical max (n -1, where n is the number of boys).

In [70]:
races = buildRaces(boys, 2)
print("The following {} races will be scheduled.".format(len(races)))
races

Counter({'Zach': 2, 'Ryan': 2, 'Noah': 2, 'Torrin': 2, 'Edgar': 2, 'Braxton': 2, 'Danny': 2, 'David': 2, 'Stirling': 2, 'Michael': 2, 'Ian': 2})
Counter({'Edgar': 2, 'Zach': 2, 'Ryan': 2, 'Noah': 2, 'Torrin': 2, 'Michael': 2, 'Danny': 2, 'David': 2, 'Stirling': 2, 'Braxton': 2, 'Ian': 2})
Counter({'Danny': 2, 'Zach': 2, 'Ryan': 2, 'Noah': 2, 'Torrin': 2, 'Michael': 2, 'Edgar': 2, 'David': 2, 'Stirling': 2, 'Braxton': 2, 'Ian': 2})
The following 22 races will be scheduled.


[('Michael', 'Torrin', 'Noah'),
 ('Braxton', 'Stirling', 'Ian'),
 ('Noah', 'Ryan', 'Edgar'),
 ('Ian', 'Braxton', 'David'),
 ('Danny', 'Ryan', 'Torrin'),
 ('Zach', 'Michael', 'David'),
 ('Edgar', 'Noah', 'Stirling'),
 ('Ryan', 'Ian', 'Danny'),
 ('Braxton', 'Torrin', 'Noah'),
 ('Stirling', 'Danny', 'Ian'),
 ('Noah', 'Braxton', 'Zach'),
 ('David', 'Edgar', 'Ryan'),
 ('Zach', 'Danny', 'Michael'),
 ('Edgar', 'Ian', 'Stirling'),
 ('Danny', 'David', 'Torrin'),
 ('Michael', 'Noah', 'Braxton'),
 ('Stirling', 'Zach', 'Edgar'),
 ('David', 'Michael', 'Ryan'),
 ('Torrin', 'Stirling', 'Danny'),
 ('Ian', 'Zach', 'Michael'),
 ('Torrin', 'David', 'Braxton'),
 ('Ryan', 'Edgar', 'Zach')]

Copy and paste everything above after the "Out[N]:", beginning with the open square bracket "[" through the closing square bracket. Paste below after the equals sign. As the races run, change the -1s to the lane number of the winner. So, for race (A,B), if A wins, then make sure that line of the outcomes reads (A,B,0)

In [73]:
outcomes = [('Michael', 'Torrin', 'Noah',1,0),
 ('Braxton', 'Stirling', 'Ian',0,2),
 ('Noah', 'Ryan', 'Edgar',2,1),
 ('Ian', 'Braxton', 'David',1,0),
 ('Danny', 'Ryan', 'Torrin',1,0),
 ('Zach', 'Michael', 'David',0,1),
 ('Edgar', 'Noah', 'Stirling',0,1),
 ('Ryan', 'Ian', 'Danny',2,0),
 ('Braxton', 'Torrin', 'Noah',0,1),
 ('Stirling', 'Danny', 'Ian',2,1),
 ('Noah', 'Braxton', 'Zach',1,2),
 ('David', 'Edgar', 'Ryan',0,2),
 ('Zach', 'Danny', 'Michael',1,2),
 ('Edgar', 'Ian', 'Stirling',0,1),
 ('Danny', 'David', 'Torrin',0,2),
 ('Michael', 'Noah', 'Braxton',2,0),
 ('Stirling', 'Zach', 'Edgar',1,2),
 ('David', 'Michael', 'Ryan',1,2),
 ('Torrin', 'Stirling', 'Danny',2,0),
 ('Ian', 'Zach', 'Michael',1,0),
 ('Torrin', 'David', 'Braxton',2,0),
 ('Ryan', 'Edgar', 'Zach',2,0)]

In [74]:
winCounter = Counter()
secondCounter = Counter()
laneCounter = Counter()
laneCounter2 = Counter()
for outcome in outcomes:
    winner = outcome[outcome[-2]]
    second = outcome[outcome[-1]]
    laneCounter[outcome[-2]] += 1
    laneCounter[outcome[-1]] += 1
    winCounter[winner] += 1
    secondCounter[second] += 1
print(winCounter)
print(secondCounter)

Counter({'Braxton': 6, 'Zach': 4, 'Danny': 4, 'Edgar': 3, 'Ryan': 1, 'Torrin': 1, 'David': 1, 'Michael': 1, 'Ian': 1})
Counter({'Ryan': 5, 'Torrin': 4, 'Michael': 4, 'Ian': 4, 'Danny': 2, 'Zach': 1, 'Noah': 1, 'Edgar': 1})


In [76]:
scores = Counter()
for k,v in winCounter.items():
    scores[k] = v *2
scores

Counter({'Braxton': 12, 'Danny': 8, 'Zach': 8, 'Edgar': 6, 'Ryan': 2, 'Torrin': 2, 'David': 2, 'Michael': 2, 'Ian': 2})

In [77]:
scores.update(secondCounter)
scores

Counter({'Braxton': 12, 'Danny': 10, 'Zach': 9, 'Edgar': 7, 'Ryan': 7, 'Torrin': 6, 'Michael': 6, 'Ian': 6, 'David': 2, 'Noah': 1})

In [78]:
laneCounter

Counter({0: 16, 1: 14, 2: 14})